In [1]:
import codecs
from tqdm import tqdm
import pandas as pd
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
import numpy as np
import xgboost as xgb

import re
import jieba

In [2]:
data=pd.read_excel('data/复旦大学中文文本分类语料.xlsx','sheet1') 

# 提取汉字
def find_chinese(file):
    pattern = re.compile(r'[^\u4e00-\u9fa5]')
    chinese = re.sub(pattern, '', file)
    return chinese

data['文本分词']= data['正文'].apply(lambda x: find_chinese(x))

In [3]:
jieba.enable_parallel() #并行分词开启
data['文本分词'] = data['文本分词'].apply(lambda i:jieba.cut(i) )
data['文本分词'] =[' '.join(i) for i in data['文本分词']]

Building prefix dict from the default dictionary ...
Loading model from cache /tmp/jieba.cache
Loading model cost 0.934 seconds.
Prefix dict has been built succesfully.


In [4]:
data.head()

,分类,正文,文本分词
0,艺术,﻿【 文献号 】1-2432\n【原文出处】出版发行研究\n【原刊地名】京\n【原刊期号】1...,文献号 原文 出处 出版发行 研究 原刊 地名 京原 刊期 号 原刊 页 号 分类号 分类 ...
1,艺术,﻿【 文献号 】1-2435\n【原文出处】扬州师院学报：社科版\n【原刊期号】199504...,文献号 原文 出处 扬州 师院 学报 社科 版原 刊期 号 原刊 页 号 分类号 分类 名 ...
2,艺术,﻿【 文献号 】1-2785\n【原文出处】南通师专学报：社科版\n【原刊期号】199503...,文献号 原文 出处 南通 师专 学报 社科 版原 刊期 号 原刊 页 号 分类号 分类 名 ...
3,艺术,﻿【 文献号 】1-3021\n【原文出处】社会科学战线\n【原刊地名】长春\n【原刊期号】...,文献号 原文 出处 社会科学 战线 原刊 地名 长春 原刊 期号 原刊 页 号 分类号 分类...
4,艺术,﻿【 文献号 】1-3062\n【原文出处】上海文化\n【原刊期号】199505\n【原刊页...,文献号 原文 出处 上海 文化 原刊 期号 原刊 页 号 分类号 分类 名 文艺理论 作者 ...


In [5]:
lbl_enc = preprocessing.LabelEncoder()
y = lbl_enc.fit_transform(data.分类.values)

In [6]:
xtrain, xvalid, ytrain, yvalid = train_test_split(data.文本分词.values, y, 
                                                  stratify=y, 
                                                  random_state=42, 
                                                  test_size=0.1, shuffle=True)


In [7]:
X=data['文本分词']
X=[i.split() for i in X]

In [8]:
# 训练word2vec词向量:
import gensim

model = gensim.models.Word2Vec(X,min_count =5,window =8,size=100)   # X是经分词后的文本构成的list，也就是tokens的列表的列表
embeddings_index = dict(zip(model.wv.index2word, model.wv.vectors))

print('Found %s word vectors.' % len(embeddings_index))

Found 92096 word vectors.


In [9]:
#该函数会将语句转化为一个标准化的向量（Normalized Vector）
#import nltk
#nltk.download('punkt')
  
def sent2vec(s):
    import jieba
    jieba.enable_parallel() #并行分词开启
    words = str(s).lower()
    #words = word_tokenize(words)
    words = jieba.lcut(words)
    words = [w for w in words if not w in stwlist]
    #words = [w for w in words if w.isalpha()]
    M = []
    for w in words:
        try:
            #M.append(embeddings_index[w])
            M.append(model[w])
        except:
            continue
    M = np.array(M)
    v = M.sum(axis=0)
    if type(v) != np.ndarray:
        return np.zeros(300)
    return v / np.sqrt((v ** 2).sum())

In [10]:
stwlist=[line.strip() for line in open('data/停用词汇总.txt',
'r',encoding='utf-8').readlines()]

In [11]:
# 对训练集和验证集使用上述函数，进行文本向量化处理
xtrain_w2v = [sent2vec(x) for x in tqdm(xtrain)]
xvalid_w2v = [sent2vec(x) for x in tqdm(xvalid)]

  1%|          | 60/8324 [01:38<4:20:52,  1.89s/it]

OSError: [Errno 24] Too many open files

In [ ]:
# 基于word2vec特征在一个简单的Xgboost模型上进行拟合
clf = xgb.XGBClassifier(nthread=10, silent=False)
clf.fit(xtrain_w2v, ytrain)
predictions = clf.predict_proba(xvalid_w2v)

print ("logloss: %0.3f " % multiclass_logloss(yvalid, predictions))

In [ ]:
# 基于word2vec特征在一个简单的Xgboost模型上进行拟合
clf = xgb.XGBClassifier(max_depth=7, n_estimators=200, colsample_bytree=0.8, 
                        subsample=0.8, nthread=10, learning_rate=0.1, silent=False)
clf.fit(xtrain_w2v, ytrain)
predictions = clf.predict_proba(xvalid_w2v)

print ("logloss: %0.3f " % multiclass_logloss(yvalid, predictions))
print(classification_report(predictions, yvalid))